<a href="https://colab.research.google.com/github/Fazlibeqir/Timski-Proekt/blob/main/YOLO8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install bing-image-downloader
!pip install ultralytics

In [18]:
import os
import pandas as pd
import shutil
import random
from bing_image_downloader import downloader
from google.colab import files
import zipfile
from ultralytics import YOLO

In [15]:
from google.colab import files
uploaded = files.upload()

Saving 0044847-241126133413365.csv to 0044847-241126133413365.csv


In [19]:
import io

In [20]:
filename = next(iter(uploaded))
df = pd.read_csv(
    io.BytesIO(uploaded[filename]),
    sep='\t',
    encoding='utf-8',
    engine='python',
    on_bad_lines='skip')
species_list = df['scientificName'].dropna().unique().tolist()
species_list = sorted(species_list)[:10]
print("Selected species:")
for s in species_list:
  print(s)

Selected species:
? agilis Ashmead, 1905
? flavomarginatus Rüppell, 1830
? fulvipenne Smith, 1857
? maculipennis Smith, 1859
? medecassus Saussure, 1887
? nigripennis Szepligeti, 1914
? nitidulum Neumayr, 1873
? perpulchra Packard, 1873
Abia nigricornis Leach, 1817
Abispa splendida australis Smith, 1857


In [21]:
base_dir = "/content/insect_classification"
for split in ["train", "val","test"]:
  for name in species_list:
    folder = os.path.join(base_dir, split, name.replace(" ","_"))
    os.makedirs(folder, exist_ok=True)

In [22]:
def download_images(query, limit=100):
  downloader.download(query, limit=limit, output_dir='downloads', adult_filter_off=True, force_replace=False, timeout=60)

In [23]:
for name in species_list:
  query = name
  print(f"Downloading images for")
  download_images(query, limit=100)

Streaming output truncated to the last 5000 lines.
[%] Downloading Image #85 from https://c8.alamy.com/comp/2AM7A74/hardwickes-science-gossip-an-illustrated-medium-of-interchange-and-gossip-for-students-and-lovers-of-nature-w-i-j-fig-219-anopheles-maculipennis-male-and-female-specific-identification-and-received-the-informationthat-they-were-only-a-species-of-gnat-anophelesmaculipennis-previously-known-as-british-and-of-which-we-figure-the-male-and-female-drawn-fromthe-woolwich-specimens-so-far-the-evidencebreaks-down-and-we-are-still-sceptical-about-mos-quitoes-in-england-4-notice-by-professor-west-wood-in-the-gardeners-chronicle-of-august-8thfavours-this-scepticism-and-is-to-the-2AM7A74.jpg
[%] File Downloaded !



[!!]Indexing page: 37

[%] Indexed 35 Images on Page 37.


[%] Downloading Image #86 from https://0.academia-photos.com/attachment_thumbnails/91843656/mini_magick20221002-1-ossjpu.png?1664683577
[%] File Downloaded !



[!!]Indexing page: 38

[%] Indexed 35 Images on Page 

In [24]:
def distrubute_images(class_name):
  class_folder = f"downloads/{class_name}"
  images = [ img for img in os.listdir(class_folder) if img.endswith(".jpg") or img.endswith(".jpeg") or img.endswith(".png") ]
  random.shuffle(images)

  train_cutoff = int(len(images)*0.7)
  val_cutoff = int(len(images)*0.9)

  for i, img in enumerate(images):
    if i < train_cutoff:
      split="train"
    elif i< val_cutoff:
      split="val"
    else:
      split="test"

    src = os.path.join(class_folder, img)
    dst = os.path.join(base_dir, split, class_name.replace(" ","_"),img)
    shutil.copyfile(src, dst)

In [25]:
for name in species_list:
  distrubute_images(name)

print("Dataset prepared in `/content/insect_classification`")

Dataset prepared in `/content/insect_classification`


In [29]:
!zip -r insect_classification.zip /content/insect_classification

  adding: content/insect_classification/ (stored 0%)
  adding: content/insect_classification/test/ (stored 0%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/ (stored 0%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_46.png (deflated 2%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_96.jpg (deflated 1%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_43.jpg (deflated 0%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_100.jpg (deflated 8%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_12.jpg (deflated 6%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_16.jpg (deflated 3%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_72.jpg (deflated 18%)
  adding: content/insect_classification/test/?_medecassus_Saussure,_1887/Image_67.jpg (deflated 0%)
 

In [30]:
from google.colab import files
files.download("insect_classification.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
data_yaml = f"""
train: /content/insect_classification/train
val: /content/insect_classification/val
test: /content/insect_classification/test

nc: {len(species_list)}  # number of classes
names: {species_list}  # list of class names
"""

In [32]:
with open("/content/insect_classification/data.yaml", "w") as file:
    file.write(data_yaml)

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


in manifest we need to change the path to absolute -> /content/insect_roboflowDataset/test/images like this

In [48]:
model = YOLO("yolov8s-cls.pt")
model.train(data="/content/datasets/content/insect_classification", epochs=1, imgsz=320, batch=4, device=0,verbose=False, name="insect_cls_yolov8")

Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/content/insect_classification, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=insect_cls_yolov816, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

train: Scanning /content/datasets/content/insect_classification/train... 679 images, 0 corrupt: 100%|██████████| 679/679 [00:00<00:00, 2282.06it/s]

train: New cache created: /content/datasets/content/insect_classification/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 382.5±87.1 MB/s, size: 119.1 KB)


val: Scanning /content/datasets/content/insect_classification/val... 195 images, 0 corrupt: 100%|██████████| 195/195 [00:00<00:00, 3115.71it/s]

...

               classes   top1_acc   top5_acc:  24%|██▍       | 6/25 [00:01<00:06,  2.91it/s]
  0%|          | 0.00/22.2M [00:00<?, ?B/s]
 46%|████▌     | 10.1M/22.2M [00:00<00:00, 85.6MB/s]
100%|██████████| 22.2M/22.2M [00:00<00:00, 55.0MB/s]
               classes   top1_acc   top5_acc: 100%|██████████| 25/25 [00:04<00:00,  6.10it/s]


                   all      0.308      0.733

1 epochs completed in 0.008 hours.
Optimizer stripped from runs/classify/insect_cls_yolov816/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/insect_cls_yolov816/weights/best.pt, 10.3MB

Validating runs/classify/insect_cls_yolov816/weights/best.pt...
Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,088,010 parameters, 0 gradients, 12.5 GFLOPs
train: /content/datasets/content/insect_classification/train... found 679 images in 10 classes ✅ 
val: /content/datasets/content/insect_classification/val... found 195 images in 10 classes ✅ 
test: /content/datasets/content/insect_classification/test... found 100 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 25/25 [00:03<00:00,  7.22it/s]


                   all      0.308      0.733
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/insect_cls_yolov816


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c8a4097a190>
curves: []
curves_results: []
fitness: 0.5205128341913223
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.3076923191547394, 'metrics/accuracy_top5': 0.7333333492279053, 'fitness': 0.5205128341913223}
save_dir: PosixPath('runs/classify/insect_cls_yolov816')
speed: {'preprocess': 0.19679723077271588, 'inference': 1.8553578666587083, 'loss': 0.001818189733827379, 'postprocess': 0.0024973025807785466}
task: 'classify'
top1: 0.3076923191547394
top5: 0.7333333492279053

In [49]:
model.val()

Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,088,010 parameters, 0 gradients, 12.5 GFLOPs
train: /content/datasets/content/insect_classification/train... found 679 images in 10 classes ✅ 
val: /content/datasets/content/insect_classification/val... found 195 images in 10 classes ✅ 
test: /content/datasets/content/insect_classification/test... found 100 images in 10 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1286.9±888.9 MB/s, size: 119.1 KB)


val: Scanning /content/datasets/content/insect_classification/val... 195 images, 0 corrupt: 100%|██████████| 195/195 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 49/49 [00:04<00:00, 11.09it/s]


                   all      0.308      0.733
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/insect_cls_yolov8162


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c8a5ed25090>
curves: []
curves_results: []
fitness: 0.5205128341913223
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.3076923191547394, 'metrics/accuracy_top5': 0.7333333492279053, 'fitness': 0.5205128341913223}
save_dir: PosixPath('runs/classify/insect_cls_yolov8162')
speed: {'preprocess': 0.29899025641311594, 'inference': 3.6036540512773523, 'loss': 0.004522271788468621, 'postprocess': 0.009652169212178053}
task: 'classify'
top1: 0.3076923191547394
top5: 0.7333333492279053

In [50]:
results = model.predict(source="/content/datasets/content/insect_classification/test/Abia_nigricornis_Leach,_1817/Image_22.png", save=True)

Results saved to runs/classify/insect_cls_yolov8163


We can download the best model to used it some where

In [51]:
files.download('/content/runs/train/insect_cls_yolov8/weights/best.pt')

FileNotFoundError: Cannot find file: /content/runs/train/insect_cls_yolov8/weights/best.pt

Visualization

In [16]:
!pip install matplotlib seaborn

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [52]:
history = model.history
epochs = list(range(1, len(history['train_loss']) + 1))

AttributeError: 'ClassificationModel' object has no attribute 'history'

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(epochs, history['train_loss'], label='Training Loss', color='red')
plt.plot(epochs, history['val_loss'], label='Validation Loss', color='blue')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(epochs, history['train_acc'], label='Training Accuracy', color='green')
plt.plot(epochs, history['val_acc'], label='Validation Accuracy', color='orange')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
results = model.val()

In [ ]:
true_labels = results.metrics['labels']  # Change this to actual label data
predicted_labels = results.pred

In [ ]:
cm = confusion_matrix(true_labels, predicted_labels)


In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=species_list, yticklabels=species_list)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
image_path = "/content/insect_classification/test/Osmia_bicornis/sample.jpg" #Changeee this
results = model.predict(source=image_path)

# Display the image and prediction result
from PIL import Image
img = Image.open(image_path)
plt.imshow(img)
plt.title(f"Prediction: {results[0].names[results[0].boxes.cls[0].item()]} - Confidence: {results[0].probs[0]:.2f}")
plt.axis('off')
plt.show()